# import

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data

# load dataset

In [2]:
mnist = input_data.read_data_sets("./data/mnist/", one_hot=True) 

Extracting ./data/mnist/train-images-idx3-ubyte.gz
Extracting ./data/mnist/train-labels-idx1-ubyte.gz
Extracting ./data/mnist/t10k-images-idx3-ubyte.gz
Extracting ./data/mnist/t10k-labels-idx1-ubyte.gz


# build model

In [3]:
def mlp(x, output_dim, reuse=False):
        
    w1 = tf.get_variable("w1", [x.get_shape()[1], 1024], initializer=tf.random_normal_initializer())
    b1 = tf.get_variable("b1", [1024], initializer=tf.constant_initializer(0.0))
    w2 = tf.get_variable("w2", [1024, output_dim], initializer=tf.random_normal_initializer())
    b2 = tf.get_variable("b2", [output_dim], initializer=tf.constant_initializer(0.0))
    
    fc1 = tf.nn.relu(tf.matmul(x, w1) + b1)
    fc2 = tf.matmul(fc1, w2) + b2

    return fc2, [w1, b1, w2, b2]

def slp(x, output_dim):
    w1 = tf.get_variable("w1", [x.get_shape()[1], output_dim], initializer=tf.random_normal_initializer())
    b1 = tf.get_variable("b1", [output_dim], initializer=tf.constant_initializer(0.0))
    
    fc1 = tf.nn.relu(tf.matmul(x, w1) + b1)
    return fc1, [w1, b1]

n_batch = 32
n_label = 10
n_train = 10000
imagesize = 28
learning_rate = 0.5

x_node = tf.placeholder(tf.float32, shape=(n_batch, imagesize*imagesize))
y_node = tf.placeholder(tf.float32, shape=(n_batch, n_label))

with tf.variable_scope("MLP") as scope:
    out_m, theta_m = mlp(x_node, n_label)
           
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(out_m, y_node))
opt  = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
tr_pred = tf.nn.softmax(out_m)

test_data = mnist.test.images
test_labels = mnist.test.labels
tx = tf.constant(test_data)
ty_ = tf.constant(test_labels)

with tf.variable_scope("MLP") as scope:
    scope.reuse_variables()
    ty, _ = mlp(tx, n_label)
    
te_pred = tf.nn.softmax(ty) 

In [4]:
def accuracy(y, y_):
    return 100.0 * np.sum(np.argmax(y, 1) == np.argmax(y_, 1)) / y.shape[0]

In [5]:
saver = tf.train.Saver()

sess=tf.InteractiveSession()

init = tf.initialize_all_variables()
sess.run(init)

In [6]:
for step in xrange(n_train):
    bx, by = mnist.train.next_batch(n_batch)
    feed_dict = {x_node: bx, y_node: by}
    _, _loss, _tr_pred = sess.run([opt, loss, tr_pred], feed_dict=feed_dict)
    if step % 500 == 0:
        _accuracy = accuracy(_tr_pred, by)
        print 'step = %d, loss=%.2f, accuracy=%.2f' % (step, _loss, _accuracy)

print 'test accuracy=%.2f' % accuracy(te_pred.eval(), test_labels)

step = 0, loss=323.57, accuracy=6.25
step = 500, loss=0.92, accuracy=96.88
step = 1000, loss=0.97, accuracy=93.75
step = 1500, loss=0.73, accuracy=90.62
step = 2000, loss=0.17, accuracy=93.75
step = 2500, loss=0.42, accuracy=90.62
step = 3000, loss=0.01, accuracy=100.00
step = 3500, loss=0.07, accuracy=96.88
step = 4000, loss=0.01, accuracy=100.00
step = 4500, loss=0.13, accuracy=93.75
step = 5000, loss=0.49, accuracy=90.62
step = 5500, loss=0.02, accuracy=100.00
step = 6000, loss=0.50, accuracy=96.88
step = 6500, loss=0.03, accuracy=100.00
step = 7000, loss=0.15, accuracy=96.88
step = 7500, loss=0.02, accuracy=100.00
step = 8000, loss=0.21, accuracy=93.75
step = 8500, loss=0.04, accuracy=100.00
step = 9000, loss=0.05, accuracy=100.00
step = 9500, loss=0.05, accuracy=96.88
test accuracy=95.03


In [8]:
old_theta_m = [ p.eval() for p in theta_m] # for comparing

In [9]:
saver.save(sess, "model.ckpt")

'model.ckpt'

In [10]:
sess.close()

In [11]:
sess=tf.InteractiveSession()

# for clear
init = tf.initialize_all_variables()
sess.run(init)

In [12]:
for prm, prm_o in zip(theta_m, old_theta_m):
    p1 = prm.eval()
    p2 = prm_o
    print np.sum(p1 != p2) 

802816
1024
10240
10


In [13]:
saver.restore(sess, "model.ckpt")

In [14]:
for prm, prm_o in zip(theta_m, old_theta_m):
    p1 = prm.eval()
    p2 = prm_o
    print np.sum(p1 != p2) 

0
0
0
0


In [15]:
print 'test accuracy=%.2f' % accuracy(te_pred.eval(), test_labels)

test accuracy=95.03


In [16]:
for step in xrange(n_train):
    bx, by = mnist.train.next_batch(n_batch)
    feed_dict = {x_node: bx, y_node: by}
    _, _loss, _tr_pred = sess.run([opt, loss, tr_pred], feed_dict=feed_dict)
    if step % 500 == 0:
        _accuracy = accuracy(_tr_pred, by)
        print 'step = %d, loss=%.2f, accuracy=%.2f' % (step, _loss, _accuracy)

print 'test accuracy=%.2f' % accuracy(te_pred.eval(), test_labels)

step = 0, loss=0.00, accuracy=100.00
step = 500, loss=0.04, accuracy=96.88
step = 1000, loss=0.00, accuracy=100.00
step = 1500, loss=0.02, accuracy=100.00
step = 2000, loss=0.00, accuracy=100.00
step = 2500, loss=0.00, accuracy=100.00
step = 3000, loss=0.06, accuracy=96.88
step = 3500, loss=0.00, accuracy=100.00
step = 4000, loss=0.00, accuracy=100.00
step = 4500, loss=0.00, accuracy=100.00
step = 5000, loss=0.00, accuracy=100.00
step = 5500, loss=0.01, accuracy=100.00
step = 6000, loss=0.00, accuracy=100.00
step = 6500, loss=0.00, accuracy=100.00
step = 7000, loss=0.00, accuracy=100.00
step = 7500, loss=0.02, accuracy=100.00
step = 8000, loss=0.00, accuracy=100.00
step = 8500, loss=0.01, accuracy=100.00
step = 9000, loss=0.00, accuracy=100.00
step = 9500, loss=0.01, accuracy=100.00
test accuracy=95.73


In [17]:
sess.close()

In [ ]:
tf.reset_default_graph()